In [1]:
# -----------------------------
# 0. Imports
# -----------------------------
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv
import os

# -----------------------------
# 1. Load Bearer Token from .env
# -----------------------------
load_dotenv()

BEARER_TOKEN = os.getenv("BEARER_TOKEN")

if BEARER_TOKEN is None:
    raise ValueError("Bearer token not found. Check your .env file.")

# -----------------------------
# 2. Twitter API Client
# -----------------------------
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# -----------------------------
# 3. Fetch Tweets
# -----------------------------

query = '"education" OR "school reform" OR "student loans" OR "literacy" -is:retweet lang:en'

response = client.search_recent_tweets(
    query=query,
    max_results=100
)

if response.data is None:
    raise ValueError("No tweets returned. Try a different query.")

tweets = [tweet.text for tweet in response.data]
df = pd.DataFrame(tweets, columns=["tweet"])

# -----------------------------
# 4. Sentiment Analysis
# -----------------------------
analyzer = SentimentIntensityAnalyzer()

df["sentiment_score"] = df["tweet"].apply(
    lambda x: analyzer.polarity_scores(x)["compound"]
)

def label_sentiment(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

df["sentiment"] = df["sentiment_score"].apply(label_sentiment)

# -----------------------------
# 5. Save CSV
# -----------------------------
df.to_csv("data/social_issues.csv", index=False)

print("✅ Sentiment analysis completed successfully")


TooManyRequests: 429 Too Many Requests
Usage cap exceeded: Monthly product cap